# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 16 2022 8:45AM,253245,12,HH-38751,Hush Hush,Released
1,Dec 16 2022 8:45AM,253245,12,HH-38752,Hush Hush,Released
2,Dec 16 2022 8:45AM,253245,12,HH-38753,Hush Hush,Released
3,Dec 16 2022 8:45AM,253245,12,HH-38754,Hush Hush,Released
4,Dec 16 2022 8:45AM,253245,12,HH-38756,Hush Hush,Released
5,Dec 16 2022 8:45AM,253245,12,HH-38757,Hush Hush,Released
6,Dec 16 2022 8:45AM,253245,12,HH-38760,Hush Hush,Released
7,Dec 16 2022 8:45AM,253245,12,HH-38761,Hush Hush,Released
8,Dec 16 2022 8:45AM,253245,12,HH-38762,Hush Hush,Released
9,Dec 16 2022 8:45AM,253245,12,HH-38763,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,253240,Released,54
30,253241,Released,1
31,253243,Released,1
32,253244,Released,13
33,253245,Released,39


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253240,NaN,NaN,54.0
253241,NaN,NaN,1.0
253243,NaN,NaN,1.0
253244,NaN,NaN,13.0
253245,NaN,NaN,39.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253179,0.0,0.0,1.0
253181,0.0,0.0,1.0
253184,6.0,4.0,5.0
253186,0.0,0.0,2.0
253195,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253179,0,0,1
253181,0,0,1
253184,6,4,5
253186,0,0,2
253195,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253179,0,0,1
1,253181,0,0,1
2,253184,6,4,5
3,253186,0,0,2
4,253195,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253179,,,1
1,253181,,,1
2,253184,6,4,5
3,253186,,,2
4,253195,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 16 2022 8:45AM,253245,12,Hush Hush
39,Dec 16 2022 8:35AM,253244,19,"AdvaGen Pharma, Ltd"
52,Dec 16 2022 8:32AM,253243,19,"GCH Granules USA, Inc."
53,Dec 16 2022 8:24AM,253241,19,"GUSA Granules USA, Inc."
54,Dec 16 2022 8:19AM,253240,19,"AdvaGen Pharma, Ltd"
108,Dec 16 2022 7:56AM,253239,19,"AdvaGen Pharma, Ltd"
169,Dec 15 2022 4:39PM,253238,10,"Citieffe, Inc."
170,Dec 15 2022 4:36PM,253236,10,"Citieffe, Inc."
171,Dec 15 2022 4:34PM,253235,19,"GUSA Granules USA, Inc."
173,Dec 15 2022 4:17PM,253230,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 16 2022 8:45AM,253245,12,Hush Hush,,,39
1,Dec 16 2022 8:35AM,253244,19,"AdvaGen Pharma, Ltd",,,13
2,Dec 16 2022 8:32AM,253243,19,"GCH Granules USA, Inc.",,,1
3,Dec 16 2022 8:24AM,253241,19,"GUSA Granules USA, Inc.",,,1
4,Dec 16 2022 8:19AM,253240,19,"AdvaGen Pharma, Ltd",,,54
5,Dec 16 2022 7:56AM,253239,19,"AdvaGen Pharma, Ltd",,5,56
6,Dec 15 2022 4:39PM,253238,10,"Citieffe, Inc.",,,1
7,Dec 15 2022 4:36PM,253236,10,"Citieffe, Inc.",,,1
8,Dec 15 2022 4:34PM,253235,19,"GUSA Granules USA, Inc.",,,2
9,Dec 15 2022 4:17PM,253230,10,ISDIN Corporation,,,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 16 2022 8:45AM,253245,12,Hush Hush,39,,
1,Dec 16 2022 8:35AM,253244,19,"AdvaGen Pharma, Ltd",13,,
2,Dec 16 2022 8:32AM,253243,19,"GCH Granules USA, Inc.",1,,
3,Dec 16 2022 8:24AM,253241,19,"GUSA Granules USA, Inc.",1,,
4,Dec 16 2022 8:19AM,253240,19,"AdvaGen Pharma, Ltd",54,,
5,Dec 16 2022 7:56AM,253239,19,"AdvaGen Pharma, Ltd",56,5,
6,Dec 15 2022 4:39PM,253238,10,"Citieffe, Inc.",1,,
7,Dec 15 2022 4:36PM,253236,10,"Citieffe, Inc.",1,,
8,Dec 15 2022 4:34PM,253235,19,"GUSA Granules USA, Inc.",2,,
9,Dec 15 2022 4:17PM,253230,10,ISDIN Corporation,10,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 16 2022 8:45AM,253245,12,Hush Hush,39,,
1,Dec 16 2022 8:35AM,253244,19,"AdvaGen Pharma, Ltd",13,,
2,Dec 16 2022 8:32AM,253243,19,"GCH Granules USA, Inc.",1,,
3,Dec 16 2022 8:24AM,253241,19,"GUSA Granules USA, Inc.",1,,
4,Dec 16 2022 8:19AM,253240,19,"AdvaGen Pharma, Ltd",54,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 16 2022 8:45AM,253245,12,Hush Hush,39.0,NaN,NaN
1,Dec 16 2022 8:35AM,253244,19,"AdvaGen Pharma, Ltd",13.0,NaN,NaN
2,Dec 16 2022 8:32AM,253243,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
3,Dec 16 2022 8:24AM,253241,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
4,Dec 16 2022 8:19AM,253240,19,"AdvaGen Pharma, Ltd",54.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 16 2022 8:45AM,253245,12,Hush Hush,39.0,0.0,0.0
1,Dec 16 2022 8:35AM,253244,19,"AdvaGen Pharma, Ltd",13.0,0.0,0.0
2,Dec 16 2022 8:32AM,253243,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
3,Dec 16 2022 8:24AM,253241,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
4,Dec 16 2022 8:19AM,253240,19,"AdvaGen Pharma, Ltd",54.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3291839,47.0,4.0,10.0
12,759652,40.0,1.0,0.0
15,506449,16.0,0.0,0.0
16,506360,2.0,0.0,0.0
18,253227,0.0,1.0,0.0
19,1772649,128.0,5.0,0.0
21,506391,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3291839,47.0,4.0,10.0
1,12,759652,40.0,1.0,0.0
2,15,506449,16.0,0.0,0.0
3,16,506360,2.0,0.0,0.0
4,18,253227,0.0,1.0,0.0
5,19,1772649,128.0,5.0,0.0
6,21,506391,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,47.0,4.0,10.0
1,12,40.0,1.0,0.0
2,15,16.0,0.0,0.0
3,16,2.0,0.0,0.0
4,18,0.0,1.0,0.0
5,19,128.0,5.0,0.0
6,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,47.0
1,12,Released,40.0
2,15,Released,16.0
3,16,Released,2.0
4,18,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,21
Status,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,4.0,1.0,0.0,0.0,1.0,5.0,1.0
Released,47.0,40.0,16.0,2.0,0.0,128.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,1.0,0.0,0.0,1.0,5.0,1.0
2,Released,47.0,40.0,16.0,2.0,0.0,128.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,4.0,1.0,0.0,0.0,1.0,5.0,1.0
2,Released,47.0,40.0,16.0,2.0,0.0,128.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()